In [1]:
!nvidia-smi

Wed Apr 12 09:01:18 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install nltk
!pip install transformers
!pip install shutup
!pip install pytorch-lightning
!pip install torchMetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 86.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 76.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.6/718.6 kB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import numpy as np
import pandas as pd
import re
import nltk
import torch
import torch.nn as nn
import pytorch_lightning as pl
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from torch.utils.data import Dataset, DataLoader, random_split
from pylab import rcParams
from tqdm.auto import tqdm
from transformers import BertTokenizer, AdamW, get_linear_schedule_with_warmup
#from torchmetrics.functional import f1, accuracy
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix
from matplotlib import rc

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
pl.seed_everything(RANDOM_SEED)

INFO:lightning_fabric.utilities.seed:Global seed set to 42


42

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [7]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/honors_data

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/honors_data


In [23]:
combined = pd.read_csv('combined_100.csv',index_col=0)
combined.reset_index(drop=True,inplace=True)
combined = combined.rename(columns={'full-text':'full_text'})
combined.head()

,id,findings,impression,full_text,image_id,label
0,317,Normal heart size. Normal mediastinal silhouet...,No acute abnormality.,Normal heart size. Normal mediastinal silhouet...,NLMCXR_png/CXR317_IM-1493-1001.png,0
1,2094,The lungs are clear without evidence of focal ...,No radiographic evidence of acute cardiopulmon...,The lungs are clear without evidence of focal ...,NLMCXR_png/CXR2094_IM-0724-1001.png,1
2,752,The lungs appear clear. Scattered calcified gr...,No acute cardiopulmonary disease,The lungs appear clear. Scattered calcified gr...,NLMCXR_png/CXR752_IM-2305-1001.png,1
3,3872,The cardiac contours are normal. The lungs are...,No acute process.,The cardiac contours are normal. The lungs are...,NLMCXR_png/CXR3872_IM-1964-1001.png,1
4,2745,Normal heart size and mediastinal contours. Cl...,No acute cardiopulmonary process. .,Normal heart size and mediastinal contours. Cl...,NLMCXR_png/CXR2745_IM-1197-1001.png,1


In [8]:
with open('visual_embeds.pkl', 'rb') as f:
  visual_embeds = pickle.load(f)

In [10]:
print(len(visual_embeds_100))
visual_embeds[0].shape

100


torch.Size([100, 1024])

In [11]:
from transformers import VisualBertModel, VisualBertConfig

configuration = VisualBertConfig.from_pretrained('uclanlp/visualbert-vqa', visual_embedding_dim=1024)
model = VisualBertModel(configuration)
model = model.to(device)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [12]:
for i in range(len(visual_embeds_100)):
  visual_embeds_100[i] = visual_embeds_100[i].to('cpu')

In [19]:
MAX_LEN=64

In [35]:
class RadiologyDataset(Dataset):

  def __init__(self, data: pd.DataFrame, tokenizer: BertTokenizer, max_len: int, visual_embeds):
    self.tokenizer = tokenizer
    self.data = data
    self.max_len = max_len
    self.visual_embeds = visual_embeds
  
  def __len__(self):
    return len(self.data)

  def __getitem__(self, index):

    data_row = self.data.iloc[index]
    text = data_row.full_text
    labels = data_row.label

    tokens = self.tokenizer.encode_plus(
      text,
      add_special_tokens=True,
      max_length=self.max_len,
      return_token_type_ids=False,
      padding="max_length",
      truncation=True,
      return_attention_mask=True,
      return_tensors='pt',
    )

    input_ids = torch.tensor(tokens["input_ids"]).flatten()
    attention_mask = torch.tensor(tokens["attention_mask"]).flatten()

    visual_embedding = self.visual_embeds[index].to('cpu')
    visual_attention_mask = torch.ones(visual_embedding.shape[:-1], dtype=torch.float)
    visual_token_type_ids = torch.ones(visual_embedding.shape[:-1], dtype=torch.long) #???

    return dict(
      input_ids=input_ids,
      attention_mask=attention_mask,
      visual_embedding=visual_embedding,
      visual_attention_mask=visual_attention_mask,
      visual_token_type_ids=visual_token_type_ids,
      labels=torch.tensor(labels).float()
    )

In [36]:
dataset = RadiologyDataset(
  combined,
  tokenizer,
  MAX_LEN,
  visual_embeds_100
)

In [37]:
train_dataset, val_dataset = train_test_split(dataset, test_size=0.10)

<ipython-input-35-73d0c649e0f3>:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(tokens["input_ids"]).flatten()
<ipython-input-35-73d0c649e0f3>:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(tokens["attention_mask"]).flatten()


In [38]:
class DataModule(pl.LightningDataModule):

  def __init__(self, train_dataset, val_dataset, tokenizer, batch_size=32, max_len=64):
    super().__init__()
    self.batch_size = batch_size
    # self.df = df
    self.train_dataset = train_dataset
    self.val_dataset = val_dataset
    self.tokenizer = tokenizer
    self.max_len = max_len
    # self.visual_embeds_mami = visual_embeds_mami
  
  # def setup(self, stage=None):
  #   self.dataset = MAMIDataset(self.df, self.tokenizer, self.max_len, self.visual_embeds_mami)
  #   self.train_dataset, self.val_dataset = train_test_split(self.dataset, test_size=0.1)
  
  def train_dataloader(self):
    return DataLoader(
        self.train_dataset,
        batch_size = self.batch_size,
        shuffle=True,
        num_workers=3
    )
  
  def val_dataloader(self):
    return DataLoader(
        self.val_dataset,
        batch_size = self.batch_size,
        num_workers=3
    )
  
  def test_dataloader(self):
    return DataLoader(
        self.val_dataset,
        batch_size = self.batch_size,
        num_workers=3
    )

In [39]:
class Classifier(pl.LightningModule):
  
  def __init__(self, n_classes, n_training_steps=None, n_warmup_steps=None):
    super().__init__()
    self.configuration = VisualBertConfig.from_pretrained('uclanlp/visualbert-vqa-coco-pre', visual_embedding_dim=1024)
    self.model = VisualBertModel(configuration)
    self.n_training_steps = n_training_steps
    self.n_warmup_steps = n_warmup_steps
    self.criterion = nn.CrossEntropyLoss()
    self.dropout = nn.Dropout(0.2)
    self.classifier = nn.Linear(self.model.config.hidden_size, n_classes)
  
  
  def forward(self, input_ids, attention_mask, visual_embeds, visual_attention_mask, visual_token_type_ids, labels=None):
    output = self.model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        visual_embeds=visual_embeds,
                        visual_attention_mask=visual_attention_mask,
                        visual_token_type_ids=visual_token_type_ids)
    
    # output = self.dropout(output.pooler_output)
    output = self.classifier(output.pooler_output)

    loss = 0
    if labels is not None:
      loss = self.criterion(output, labels)

    return loss, output
  
  def training_step(self, batch, batch_idx):
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    visual_embeds = batch['visual_embedding'].to(device)
    visual_attention_mask = batch['visual_attention_mask'].to(device)
    visual_token_type_ids = batch['visual_token_type_ids'].to(device)

    labels = batch['labels'].type(torch.LongTensor).to(device)
    
    loss, outputs = self(input_ids, attention_mask, visual_embeds, visual_attention_mask, visual_token_type_ids, labels)
    self.log('train_loss', loss, prog_bar=True, logger=True)

    return {"loss":loss, 'predictions':outputs, 'labels':labels}

  def validation_step(self, batch, batch_idx):
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    visual_embeds = batch['visual_embedding']
    visual_attention_mask = batch['visual_attention_mask']
    visual_token_type_ids = batch['visual_token_type_ids'].to(device)
    labels = batch['labels'].type(torch.LongTensor).to(device)
    
    loss, outputs = self(input_ids, attention_mask, visual_embeds, visual_attention_mask, visual_token_type_ids, labels)
    self.log('val_loss', loss, prog_bar=True, logger=True)

    return loss
  
  def configure_optimizers(self):
    optimizer = AdamW(self.parameters(), lr=3e-5)

    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=self.n_warmup_steps,
        num_training_steps=self.n_training_steps
    )

    return dict(
        optimizer=optimizer,
        lr_scheduler=dict(
            scheduler=scheduler,
            interval='step'
        )
    )

In [40]:
n_classes = 2
N_EPOCHS = 2
BATCH_SIZE = 40

In [41]:
steps_per_epoch = 8000 // BATCH_SIZE
total_training_steps = steps_per_epoch * N_EPOCHS

In [42]:
warmup_steps = total_training_steps // 10
warmup_steps, total_training_steps

(40, 400)

In [43]:
model = Classifier(
  n_classes=n_classes,
  n_training_steps=total_training_steps,
  n_warmup_steps=0,
).to(device)

In [44]:
checkpoint_callback = ModelCheckpoint(
  dirpath="checkpoints",
  filename="best-checkpoint",
  save_top_k=1,
  verbose=True,
  monitor="val_loss",
  mode="min",
)

In [45]:
logger = TensorBoardLogger("lightning_logs", name="memes-text")
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=3)

In [48]:
from sklearn.model_selection import KFold
import sklearn

dataset = RadiologyDataset(
  combined,
  tokenizer,
  MAX_LEN,
  visual_embeds_100
)

train_dataset, val_dataset = train_test_split(dataset, test_size=0.1, random_state=RANDOM_SEED, shuffle=True)

data_module = DataModule(
    train_dataset=val_dataset,
    val_dataset=val_dataset,
    tokenizer=tokenizer,
    batch_size=BATCH_SIZE,
)

trainer = pl.Trainer(
    logger=logger,
    callbacks=[early_stopping_callback, checkpoint_callback],
    max_epochs=10,
    #progress_bar_refresh_rate=10
)

trainer.fit(model, data_module)

<ipython-input-35-73d0c649e0f3>:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(tokens["input_ids"]).flatten()
<ipython-input-35-73d0c649e0f3>:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(tokens["attention_mask"]).flatten()
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/us

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.9/dist-packages/pytorch_lightning/loops/fit_loop.py:280: PossibleUserWarning: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

INFO:pytorch_lightning.utilities.rank_zero:Epoch 7, global step 8: 'val_loss' reached 0.57760 (best 0.57760), saving model to '/content/drive/MyDrive/honors_data/checkpoints/best-checkpoint.ckpt' as top 1


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 8, global step 9: 'val_loss' was not in top 1


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 9, global step 10: 'val_loss' was not in top 1
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.
